In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import torch
from torch.utils.data import Dataset, DataLoader

# Load the data
train_data = pd.read_csv('../data/train_data.csv')
val_data = pd.read_csv('../data/val_data.csv')

# Preprocessing function
def preprocess_data(data, encoder=None, scaler=None):
    categorical_features = ['distributor_id', 'industry', 'sku', 'category', 'movement_category']
    numerical_features = ['sales', 'avg_quarterly_sales', 'total_quarter_sales', 'prev_quarter_sales', 
                          'is_diwali', 'is_ganesh_chaturthi', 'is_gudi_padwa', 'is_eid', 
                          'is_akshay_tritiya', 'is_dussehra_navratri', 'is_onam', 'is_christmas', 'time_idx']

    # One-hot encoding for categorical features
    if encoder is None:
        encoder = OneHotEncoder(sparse_output=False)
        encoded_categorical = encoder.fit_transform(data[categorical_features])
    else:
        encoded_categorical = encoder.transform(data[categorical_features])
    
    # Scaling numerical features
    if scaler is None:
        scaler = StandardScaler()
        scaled_numerical = scaler.fit_transform(data[numerical_features])
    else:
        scaled_numerical = scaler.transform(data[numerical_features])
    
    # Combine processed features
    processed_data = np.hstack((encoded_categorical, scaled_numerical))
    
    print(f"Processed data shape: {processed_data.shape}")
    return processed_data, encoder, scaler

# Preprocess train and validation data
X_train = preprocess_data(train_data)
y_train = train_data['sales'].values  # Target variable
X_val = preprocess_data(val_data)
y_val = val_data['sales'].values  # Target variable

# Fit encoders/scalers on training data
X_train, encoder, scaler = preprocess_data(train_data)
X_val, _, _ = preprocess_data(val_data, encoder, scaler)


Processed data shape: (7000, 287)
Processed data shape: (1500, 284)
Processed data shape: (7000, 287)
Processed data shape: (1500, 287)


In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np

class TFTModel(nn.Module):
    def __init__(self, input_size):
        super(TFTModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)  # Output layer for regression

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

class SalesDataset(Dataset):
    def __init__(self, features, targets):
        self.features = features
        self.targets = targets

    def __len__(self):
        return len(self.targets)

    def __getitem__(self, idx):
        return torch.tensor(self.features[idx], dtype=torch.float32), torch.tensor(self.targets[idx], dtype=torch.float32)

# Create datasets
train_dataset = SalesDataset(X_train, y_train)
val_dataset = SalesDataset(X_val, y_val)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Initialize model, loss function, and optimizer
input_size = X_train.shape[1]
model = TFTModel(input_size)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Early stopping parameters
patience = 10  # Stop training if no improvement for 10 epochs
best_val_loss = float('inf')
epochs_no_improve = 0
early_stop = False

# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    for features, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs.squeeze(), targets)
        loss.backward()
        optimizer.step()

    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for features, targets in val_loader:
            outputs = model(features)
            val_loss += criterion(outputs.squeeze(), targets).item()
    
    val_loss /= len(val_loader)
    print(f'Epoch {epoch+1}/{num_epochs}, Validation Loss: {val_loss:.4f}')

    # Check early stopping criteria
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        epochs_no_improve = 0
        torch.save(model.state_dict(), 'best_model.pth')  # Save best model
    else:
        epochs_no_improve += 1

    if epochs_no_improve >= patience:
        print(f"Early stopping triggered at epoch {epoch+1}")
        early_stop = True
        break


Epoch 1/100, Validation Loss: 150907811.2340
Epoch 2/100, Validation Loss: 57408452.0319
Epoch 3/100, Validation Loss: 12107567.0851
Epoch 4/100, Validation Loss: 5279874.2394
Epoch 5/100, Validation Loss: 4330549.4914
Epoch 6/100, Validation Loss: 3850712.7068
Epoch 7/100, Validation Loss: 3513853.5512
Epoch 8/100, Validation Loss: 3194233.3969
Epoch 9/100, Validation Loss: 2891906.6210
Epoch 10/100, Validation Loss: 2617748.9814
Epoch 11/100, Validation Loss: 2359944.5549
Epoch 12/100, Validation Loss: 2050191.8404
Epoch 13/100, Validation Loss: 1823757.7154
Epoch 14/100, Validation Loss: 1622154.1546
Epoch 15/100, Validation Loss: 1402467.9315
Epoch 16/100, Validation Loss: 1196286.0199
Epoch 17/100, Validation Loss: 1017124.6682
Epoch 18/100, Validation Loss: 874863.4212
Epoch 19/100, Validation Loss: 733967.4885
Epoch 20/100, Validation Loss: 621109.8906
Epoch 21/100, Validation Loss: 508145.6179
Epoch 22/100, Validation Loss: 418307.9816
Epoch 23/100, Validation Loss: 359612.5046

In [17]:

# Load the best model
model.load_state_dict(torch.load('best_model.pth'))
print("Training complete. Best model loaded.")


Training complete. Best model loaded.


In [18]:
# Making predictions
model.eval()
with torch.no_grad():
    predictions = []
    for features, _ in val_loader:
        outputs = model(features)
        predictions.extend(outputs.squeeze().numpy())

# Convert predictions to a DataFrame for analysis
predictions_df = pd.DataFrame(predictions, columns=['Predicted Sales'])
predictions_df['Actual Sales'] = y_val


In [19]:

predictions_df.head(50)

,Predicted Sales,Actual Sales
0,2483.507080,2466.39
1,9418.099609,9412.45
2,7185.533691,7196.18
3,5872.748047,5875.68
4,8430.978516,8432.52
5,1293.731079,1301.18
6,31542.792969,31554.27
7,8223.617188,8225.04
8,2743.220459,2750.19
9,3393.801025,3385.62


In [22]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Switch to evaluation mode
model.eval()

# Collect predictions and actual values
y_true = predictions_df['Actual Sales']
y_pred = predictions_df['Predicted Sales']


# Convert lists to numpy arrays
y_true = np.array(y_true)
y_pred = np.array(y_pred)

# Compute metrics
mae = mean_absolute_error(y_true, y_pred)
mse = mean_squared_error(y_true, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_true, y_pred)

# Print results
print("\n📊 Model Evaluation Metrics:")
print(f"✅ MAE  = {mae:.4f}")
print(f"✅ MSE  = {mse:.4f}")
print(f"✅ RMSE = {rmse:.4f}")
print(f"✅ R² Score = {r2:.4f}")



📊 Model Evaluation Metrics:
✅ MAE  = 8.2704
✅ MSE  = 167.1193
✅ RMSE = 12.9275
✅ R² Score = 1.0000
